# Init

In [ ]:
# init
from importlib import reload
from pathlib import Path
import os
import sys
import pandas as pd

sys.path.append('..')
sys.path.append("C:/Users/gonta/D/software developer/packages")
import toolsGeneral.main as tgm
import toolsPandas.helpers as tph

def reloadPckg():
    reload(tgm)
    reload(tph)

# Import

In [115]:
# import
dataDir = Path(os.path.join(os.getcwd(), '..', 'data/normalized'))
rawFiles = [f for f in dataDir.rglob('*/*.pkl')]
print(len(rawFiles))

df_by_cntr = {
    file.parent.name:tgm.load(str(file))
    for file in rawFiles
}
print(len(df_by_cntr))

64
64


In [2]:
temp = pd.concat(df_by_cntr.values(), ignore_index=True)
len(temp)

118738

In [3]:
pd.Series(temp.values.flatten()).map(type).value_counts()

<class 'pandas._libs.missing.NAType'>    827091
<class 'str'>                            597765
Name: count, dtype: int64

# Apply fixes

## 1. osm basic fix

In [2]:
osm_basic_test = tgm.load(os.path.join('..', 'data', 'tests results', 'osm_basic_test.pkl'))
print(len(osm_basic_test))
to_delete = []
for k,v in osm_basic_test.items():
    to_delete.extend(v['missing.name'])
    to_delete.extend(v['leak'])

to_delete = set(to_delete)
print(len(to_delete))

64
292


In [59]:
tph.df_peek(tph.to_df(osm_basic_test).T)

,missing.name,leak
Albania,[],[]
Algeria,[],[]
Andorra,[],[]
Australia,[],[]
Austria,[],[52345]
Bangladesh,[14326679],"[3824588, 3825003, 3859335, 3921211, 3921222, 3921298, 3921322, 7318343, 9455570]"
Belarus,[],"[59162, 59189, 59195, 59275, 59506, 59752]"
Belgium,[],"[4217435, 7394, 7400, 7411, 7427, 7463, 7400]"
Bhutan,[],[]
Brazil,[],"[54882, 57963, 185579, 242620, 298204, 301079, 301464, 302635, 302819, 303702, 303781, 303940, 315173, 325866, 326266, 331463, 332476, 332924, 333597, 334051, 334443, 336819, 362413, 405935, 421151, 4059665, 298424, 298446]"


### * apply

In [ ]:
# delete missing names and leaked elements
df_by_cntr_fixed = {}
for k,df in df_by_cntr.items():
    df_by_cntr_fixed[k] = df[~df['id'].isin(to_delete)].copy()

In [133]:
combined_df_old = pd.concat(df_by_cntr.values(), ignore_index=True)
print(len(combined_df_old))
combined_df_new = pd.concat(df_by_cntr_fixed.values(), ignore_index=True)
print(len(combined_df_new))

118738
118559


In [135]:
print([len(set(combined_df_old['id'].to_list())),len(set(combined_df_new['id'].to_list()))])
print(len(to_delete))

[117634, 117471]
163


## 2. osm duplicates fix

In [137]:
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))

osm_dup_test = {
    file.parent.name:tgm.load(str(file))
    for file in data_dir.rglob('*/*.pkl')
}

In [142]:
to_delete_dup = []
for k,v in osm_dup_test.items():
    to_delete_dup.extend(v['duplicate.delete_elems'])

to_delete_dup = set(to_delete_dup)
len(to_delete_dup)

143

### * apply

In [143]:
# delete duplicate elements
for k,df in df_by_cntr_fixed.items():
    df_by_cntr_fixed[k] = df[~df[['id','tags.parent_id']].apply(tuple, axis=1).isin(to_delete_dup)].copy()

In [144]:
combined_df_old = pd.concat(df_by_cntr.values(), ignore_index=True)
print(len(combined_df_old))
combined_df_new = pd.concat(df_by_cntr_fixed.values(), ignore_index=True)
print(len(combined_df_new))

118738
118403


In [ ]:
print([len(set(combined_df_old['id'].to_list())), len(set(combined_df_new['id'].to_list()))])
len(to_delete.union(to_delete_dup))

[117634, 117459]


306

In [149]:
117634-306

117328

# 3. Save!

In [11]:
for k,df in df_by_cntr_fixed.items():
    path = os.path.join(os.getcwd(),f'../data/fixed/{k}/{k}_fixed.pkl')
    tgm.dump(path, df)